## Interfaz

In [ ]:
#%%writefile main.py

## Para que el agente mande la info

In [8]:
#%%writefile main.py
import boto3
from datetime import datetime
import pandas as pd
from io import BytesIO
import streamlit as st
session = boto3.client('s3',
    aws_access_key_id = st.secrets["aws"]["aws_access_key_id"],
    aws_secret_access_key = st.secrets["aws"]["aws_secret_access_key"],
     region_name='us-east-2'
)

# Especifica el nombre del bucket y la clave del archivo .pkl en S3




In [200]:
def send_status(agent_number, status, current_page, errors = None):
    
## Primero obtenemos el archivo:
    bucket_name = 's3-pernexium-report'
    key_folder = 'raw/didi/didi_agent/'
    fecha = datetime.now()
    fecha_actual = fecha.strftime("%Y-%m-%d")
    
    key_final_prospectos = f'{key_folder}{fecha_actual}/logs_agent_{agent_number}.csv'
    
    try:
        response = session.get_object(Bucket=bucket_name, Key=key_final_prospectos)
        columnas_bytes = response['Body'].read()
        data = pd.read_csv(BytesIO(columnas_bytes))
    except:
        data = None
    
    
    # Si ya hay uno, se hace un append
    fecha_hora_actual = fecha.strftime("%Y-%m-%d %H:%M:%S")
    payload = [agent_number, fecha_hora_actual, status, current_page, errors]
    
    data_ = pd.DataFrame(payload).T
    data_.columns = ["agent_number", "last_update", "last_status", "current_page", "errors"]
    
    if data is not None:
        data = pd.concat([data, data_])
    else:
        data = data_
        
    # Convertir DataFrame a CSV en memoria
    csv_buffer = data.to_csv(index=False)
    
    # Subir el archivo CSV a S3 directamente desde la memoria
    response = session.put_object(Bucket=bucket_name, Key=key_final_prospectos, Body=csv_buffer)
    if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        print("Enviado correctamente")
    else:
        print("Error en la carga")

    return data

In [4]:
## Sin pandas
import csv
from io import StringIO, BytesIO
from datetime import datetime

def send_status(agent_number, status, current_page, errors=None):
    # Primero obtenemos el archivo:
    bucket_name = 's3-pernexium-report'
    key_folder = 'raw/didi/didi_agent/'
    fecha = datetime.now()
    fecha_actual = fecha.strftime("%Y-%m-%d")
    
    key_final_prospectos = f'{key_folder}{fecha_actual}/logs_agent_{agent_number}.csv'
    
    try:
        response = session.get_object(Bucket=bucket_name, Key=key_final_prospectos)
        columnas_bytes = response['Body'].read()
        data = columnas_bytes.decode('utf-8')
        rows = list(csv.reader(StringIO(data)))
    except:
        rows = []

    # Si ya hay uno, se hace un append
    fecha_hora_actual = fecha.strftime("%Y-%m-%d %H:%M:%S")
    payload = [agent_number, fecha_hora_actual, status, current_page, errors]
    
    # Crear CSV en memoria
    output = StringIO()
    writer = csv.writer(output)
    
    if rows:
        writer.writerows(rows)  # Escribe las filas existentes
    else:
        # Definir las columnas si no hay datos
        writer.writerow(["agent_number", "last_update", "last_status", "current_page", "errors"])
    
    writer.writerow(payload)  # Añadir la nueva fila
    
    csv_buffer = output.getvalue()
    
    # Subir el archivo CSV a S3 directamente desde la memoria
    response = session.put_object(Bucket=bucket_name, Key=key_final_prospectos, Body=csv_buffer)
    if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        print("Enviado correctamente")
    else:
        print("Error en la carga")

    return rows


In [14]:
send_status(agent_number = 3, status = 'Finished', current_page = '51/68', errors = None)

Enviado correctamente


[['agent_number', 'last_update', 'last_status', 'current_page', 'errors'],
 ['3', '2024-08-26 17:08:27', 'Running', '50/68', ''],
 ['3', '2024-08-26 17:08:52', 'Finished', '50/68', '']]

In [144]:
try: 
    int(1)
except Exception as e:
    send_status(agent_number = 2, status = 'Todo bien', current_page = '20/20', errors = "Llamen a Dios\n"+str(e))
    raise e

## Para el servidor que la recibe

In [148]:
%%writefile main.py
import boto3
from datetime import datetime
import pandas as pd
from io import BytesIO
import streamlit as st
import pytz


session = boto3.client('s3',
    aws_access_key_id = st.secrets["aws"]["aws_access_key_id"],
    aws_secret_access_key = st.secrets["aws"]["aws_secret_access_key"],
     region_name='us-east-2'
)

# Especifica el nombre del bucket y la clave del archivo .pkl en S3
def get_data(fecha_buscar):
    data_general = pd.DataFrame()
    bucket_name = 's3-pernexium-report'
    key_folder = f'raw/didi/didi_agent/{fecha_buscar}/'
    
    response = session.list_objects_v2(Bucket=bucket_name, Prefix=key_folder)
        
    if 'Contents' in response:
        # Itera sobre cada archivo en el folder
        for obj in response['Contents']:
            key = obj['Key']
            print(f"Leyendo el archivo: {key}")
    
            # Obtén el objeto desde S3
            file_response = session.get_object(Bucket=bucket_name, Key=key)
            
            # Lee el contenido del archivo en bytes
            columnas_bytes = file_response['Body'].read()
    
            # Usa BytesIO para leer el contenido como un DataFrame de pandas
            df = pd.read_csv(BytesIO(columnas_bytes))
    
            # Realiza cualquier operación que necesites con el DataFrame
            data_general = pd.concat([data_general, df])
    else:
        print(f"No se encontraron archivos en el folder {key_folder} del bucket {bucket_name}.")
        return None

    data_general_raw = data_general.copy()
    data_general_raw["page"] = data_general_raw.current_page.apply(lambda x: int(x.split("/")[0]))
    data_general = data_general.sort_values(by = "last_update", ascending=False).drop_duplicates(subset = ["agent_number"], keep="first")
    data_general['progress'] = data_general.current_page.apply(lambda x: int(x.split("/")[0]) / int(x.split("/")[1]))

    data_general = data_general[['agent_number', 'last_update', 'last_status', 'current_page', 'progress', 'errors']]
    data_general = data_general.sort_values(by = 'agent_number')
    return data_general, data_general_raw

st.set_page_config(
    page_title="Pernexium Agentes Automáticos",
    page_icon="./img/logo_pernexium.png"  # Puedes usar una ruta local o una URL
)

st.header("Interfaz de control para agentes automáticos")

mexico_city_tz = pytz.timezone('America/Mexico_City')

# Obtén la fecha y hora actual en la zona horaria de Ciudad de México
hoy = datetime.now(mexico_city_tz).date()
#st.write(hoy)

# Selector de fechas con la fecha de hoy como valor predeterminado
col1, col2 = st.columns([9, 1])
with col1:
    fecha_seleccionada = st.date_input("Seleccione una fecha:", hoy)
with col2:
    #st.write("#")
    st.button('🔄')

data, data_raw = get_data(fecha_seleccionada)

data_raw.last_update = pd.to_datetime(data_raw.last_update)

total_gestionado = 20 * (data_raw.groupby("agent_number").page.max() - data_raw.groupby("agent_number").page.min()).sum()

agentes_corriendo = data_raw.agent_number.nunique()

gestiones_medias = int(total_gestionado/agentes_corriendo)

tiempo_medio_por_gestion = sum([data_raw.query(f"agent_number == {an}").last_update.diff().mean().total_seconds() / 20 for an in range(1, agentes_corriendo+1)])/ agentes_corriendo

gestiones_en_ocho_horas = (9*60*60) / tiempo_medio_por_gestion

    
if data is None:
    st.warning("No hay información para la fecha seleccionada")
else:
    st.data_editor(data, disabled = True, 
                   column_config={
                    "progress": st.column_config.ProgressColumn(
                        "Progress",
                        help="Progreso",
                        #format="%f",
                        min_value=0,
                        max_value=1,
                    ),
                },
                hide_index=True,)

col1, col2  = st.columns(2)
col1.metric(label = "Total de Cuentas gestionadas en el día", value = str(total_gestionado))

col2.metric(label = "Promedio de cuentas por agente en jornada", value = f"{gestiones_en_ocho_horas:.0f}", delta = f'{gestiones_en_ocho_horas - 350:.0f}')


Overwriting main.py


In [101]:
data_general.columns

Index(['agent_number', 'last_update', 'last_status', 'current_page', 'errors',
       'progress'],
      dtype='object')

In [77]:
data_general = data_general.sort_values(by = "last_update", ascending=False).drop_duplicates(subset = ["agent_number"], keep="first")

In [14]:
import boto3
from datetime import datetime
import pandas as pd
from io import BytesIO
import streamlit as st
import pytz

session = boto3.client('s3',
    aws_access_key_id = st.secrets["aws"]["aws_access_key_id"],
    aws_secret_access_key = st.secrets["aws"]["aws_secret_access_key"],
     region_name='us-east-2'
)

# Especifica el nombre del bucket y la clave del archivo .pkl en S3
def get_data(fecha_buscar):
    data_general = pd.DataFrame()
    bucket_name = 's3-pernexium-report'
    key_folder = f'raw/didi/didi_agent/{fecha_buscar}/'
    
    response = session.list_objects_v2(Bucket=bucket_name, Prefix=key_folder)
        
    if 'Contents' in response:
        # Itera sobre cada archivo en el folder
        for obj in response['Contents']:
            key = obj['Key']
            print(f"Leyendo el archivo: {key}")
    
            # Obtén el objeto desde S3
            file_response = session.get_object(Bucket=bucket_name, Key=key)
            
            # Lee el contenido del archivo en bytes
            columnas_bytes = file_response['Body'].read()
    
            # Usa BytesIO para leer el contenido como un DataFrame de pandas
            df = pd.read_csv(BytesIO(columnas_bytes))
    
            # Realiza cualquier operación que necesites con el DataFrame
            data_general = pd.concat([data_general, df])
    else:
        print(f"No se encontraron archivos en el folder {key_folder} del bucket {bucket_name}.")
        return None

    data_general_raw = data_general.copy()
    data_raw["page"] = data_raw.current_page.apply(lambda x: int(x.split("/")[0]))
    data_general = data_general.sort_values(by = "last_update", ascending=False).drop_duplicates(subset = ["agent_number"], keep="first")
    data_general['progress'] = data_general.current_page.apply(lambda x: int(x.split("/")[0]) / int(x.split("/")[1]))

    data_general = data_general[['agent_number', 'last_update', 'last_status', 'current_page', 'progress', 'errors']]
    data_general = data_general.sort_values(by = 'agent_number')
    return data_general, data_general_raw

st.set_page_config(
    page_title="Pernexium Agentes Automáticos",
    page_icon="./img/logo_pernexium.png"  # Puedes usar una ruta local o una URL
)

st.header("Interfaz de control para agentes automáticos")

mexico_city_tz = pytz.timezone('America/Mexico_City')

# Obtén la fecha y hora actual en la zona horaria de Ciudad de México
hoy = datetime.now(mexico_city_tz).date()

data, data_raw = get_data(hoy)


Leyendo el archivo: raw/didi/didi_agent/2024-08-30/logs_agent_1.csv
Leyendo el archivo: raw/didi/didi_agent/2024-08-30/logs_agent_10.csv
Leyendo el archivo: raw/didi/didi_agent/2024-08-30/logs_agent_2.csv
Leyendo el archivo: raw/didi/didi_agent/2024-08-30/logs_agent_3.csv
Leyendo el archivo: raw/didi/didi_agent/2024-08-30/logs_agent_4.csv
Leyendo el archivo: raw/didi/didi_agent/2024-08-30/logs_agent_5.csv
Leyendo el archivo: raw/didi/didi_agent/2024-08-30/logs_agent_6.csv
Leyendo el archivo: raw/didi/didi_agent/2024-08-30/logs_agent_7.csv
Leyendo el archivo: raw/didi/didi_agent/2024-08-30/logs_agent_8.csv
Leyendo el archivo: raw/didi/didi_agent/2024-08-30/logs_agent_9.csv


In [30]:
data_raw["page"] = data_raw.current_page.apply(lambda x: int(x.split("/")[0]))

In [34]:
(data_raw.groupby("agent_number").page.max() - data_raw.groupby("agent_number").page.min()).sum()

190

In [64]:
data_raw.last_update = pd.to_datetime(data_raw.last_update)

In [142]:
agentes_corriendo = 10
sum([data_raw.query(f"agent_number == {an}").last_update.diff().mean().total_seconds() / 20 for an in range(1,agentes_corriendo+1)])/ agentes_corriendo

45.17484835500001

In [162]:
# Agrupar por día
data_raw.groupby(data_raw['last_update'].dt.date).page.sum()

last_update
2024-08-30    2265
Name: page, dtype: int64